In [1]:
import pandas as pd
import os
    
datasets = {"Retail_Rocket": {"path": '/home/jy1559/Mar2025_Module/Datasets/Retail_Rocket',
                              "file_names": ['category_tree.csv', 'events.csv', 'item_properties_part1.csv', 'item_properties_part2.csv']},
            "Diginetica": {"path": '/home/jy1559/Mar2025_Module/Datasets/Diginetica',
                              "file_names": ['product-categories.csv', 'products.csv', 'train-clicks.csv', 'train-item-views.csv', 'train-purchases.csv', 'train-queries.csv']},
            "LFM-BeyMS": {"path": '/home/jy1559/Mar2025_Module/Datasets/LFM-BeyMS/dataset',
                              "file_names": ['beyms.csv', 'events.csv', 'genre_annotations.csv', 'mainstreaminess.csv', 'ms.csv', 'user_groups.csv']},
            "Beauty": {"path": '/home/jy1559/Mar2025_Module/Datasets/Amazon',
                "file_names": ['All_Beauty.jsonl', 'meta_All_Beauty.jsonl']},
            "Game": {"path": '/home/jy1559/Mar2025_Module/Datasets/Amazon',
                "file_names": ['Video_Games.jsonl', 'meta_Video_Games.jsonl']}}
dataset = datasets["Retail_Rocket"]
directory_path = dataset['path']
file_names = dataset["file_names"]
for name in file_names:
    file_path = os.path.join(directory_path, name)
    try:
        if 'csv' in file_path:
            df = pd.read_csv(file_path)
        elif 'json' in file_path:
            df = pd.read_json(file_path, lines=True)
        print(f"First 5 rows of {name}:")
        print(df.head(), "\n")
    except FileNotFoundError:
        print(f"File {name} not found in the directory {directory_path}.")
    except pd.errors.EmptyDataError:
        print(f"File {name} is empty.")
    except pd.errors.ParserError:
        print(f"Error parsing {name}. Please check the file for inconsistencies.")

First 5 rows of category_tree.csv:
   categoryid  parentid
0        1016     213.0
1         809     169.0
2         570       9.0
3        1691     885.0
4         536    1691.0 

First 5 rows of events.csv:
       timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214164     992329  view  248676            NaN
2  1433221999827     111016  view  318965            NaN
3  1433221955914     483717  view  253185            NaN
4  1433221337106     951259  view  367447            NaN 

First 5 rows of item_properties_part1.csv:
       timestamp  itemid    property                            value
0  1435460400000  460429  categoryid                             1338
1  1441508400000  206783         888          1116713 960601 n277.200
2  1439089200000  395014         400  n552.000 639502 n720.000 424566
3  1431226800000   59481         790                       n15360.000
4  1431831600000  156781         917                  

In [2]:
import pandas as pd
import json
from tqdm.auto import tqdm

# 파일 경로 (실제 경로에 맞게 수정)
part1_path = os.path.join(directory_path, dataset["file_names"][2])
part2_path = os.path.join(directory_path, dataset["file_names"][3])
category_tree_path = os.path.join(directory_path, dataset["file_names"][0])

# item_properties 파일 2개 로드 및 합치기
df_part1 = pd.read_csv(part1_path)
df_part2 = pd.read_csv(part2_path)
df_props = pd.concat([df_part1, df_part2], ignore_index=True)

# 각 (itemid, property)별로 최신 timestamp의 row만 남기기
df_props = df_props.sort_values(by=['itemid', 'property', 'timestamp'])
df_props_latest = df_props.groupby(['itemid', 'property'], as_index=False).last()

# 로드: category_tree.csv (컬럼: categoryid, parentid)
df_cat = pd.read_csv(category_tree_path)
# parentid가 NaN이면 해당 categoryid를 그대로 사용
df_cat['parentid'] = df_cat['parentid'].fillna(df_cat['categoryid'])
# 카테고리 매핑 dictionary: categoryid -> parentid
cat_dict = df_cat.set_index('categoryid')['parentid'].to_dict()

# 각 itemid별로 property 딕셔너리로 정리
def aggregate_properties(df):
    # df: 같은 itemid에 대한 DataFrame
    props = {}
    for _, row in df.iterrows():
        prop = str(row['property']).strip().lower()
        val = str(row['value']).strip()
        props[prop] = val
    return props

item_props = df_props_latest.groupby('itemid').apply(aggregate_properties).to_dict()

# 각 itemid에 대해 문장 생성
def construct_item_sentence(itemid, props):
    # categoryid와 available은 따로 처리
    sub_cat = props.get("categoryid", "").strip()
    availability = props.get("available", "").strip()
    # availability가 비어있으면 생략, 값이 있다면 "O" 또는 "X"로 변환 (여기서는 단순히 값이 '1'이면 O, 아니면 X로 가정)
    if availability:
        availability_str = "O" if availability == "1" else "X"
    else:
        availability_str = ""
    
    # 상위 카테고리: category_tree에서 sub_cat를 통해 lookup (문자열이 숫자형이라고 가정)
    parent_cat = ""
    if sub_cat:
        try:
            cat_id = int(sub_cat)
            parent_cat = str(cat_dict.get(cat_id, ""))
        except Exception:
            parent_cat = ""
    
    # 나머지 프로퍼티: categoryid와 available 제외하고 정렬하여 "Property i" 형태로 나열
    other_props = {k: v for k, v in props.items() if k not in ["categoryid", "available"]}
    # 정렬
    sorted_props = sorted(other_props.items())
    other_strs = []
    for i, (k, v) in enumerate(sorted_props, start=1):
        other_strs.append(f"Property {i} ({k}): {v}")
    
    # 문장 구성: 값이 없는 항목은 생략
    parts = []
    parts.append(f"Parent Category: {parent_cat}" if parent_cat else "Parent Category: ")
    parts.append(f"Subcategory: {sub_cat}" if sub_cat else "Subcategory: ")
    if availability_str:
        parts.append(f"Availability: {availability_str}")
    # 다른 프로퍼티는 있으면 추가
    if other_strs:
        parts.extend(other_strs)
    
    sentence = ", ".join(parts) + "."
    return sentence

# 각 itemid별 문장을 dictionary로 생성
item_sentences = {}
for itemid, props in tqdm(item_props.items()):
    sentence = construct_item_sentence(itemid, props)
    item_sentences[itemid] = sentence

# JSON 파일로 저장
with open("retailrocket_item_sentences.json", "w", encoding="utf-8") as f:
    json.dump(item_sentences, f, ensure_ascii=False, indent=2)

print("Item sentences JSON이 'retailrocket_item_sentences.json' 파일로 저장되었습니다.")


/tmp/ipykernel_3676813/204098059.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  item_props = df_props_latest.groupby('itemid').apply(aggregate_properties).to_dict()
100%|██████████| 417053/417053 [00:05<00:00, 81072.33it/s]


Item sentences JSON이 'retailrocket_item_sentences.json' 파일로 저장되었습니다.


In [4]:
import pandas as pd
import json

# 파일 경로 (실제 경로에 맞게 수정)
events_path = os.path.join(directory_path, dataset["file_names"][1])
df_events = pd.read_csv(events_path)

# timestamp를 datetime으로 변환 (Retail Rocket의 timestamp는 밀리초초 단위라고 가정)
df_events['datetime'] = pd.to_datetime(df_events['timestamp'], unit='ms')
df_events = df_events.sort_values(by=['visitorid', 'datetime'])

# 필요한 정보만 추출: (itemid, timestamp, event, transactionid)
def extract_event(row):
    return {
        "itemid": row["itemid"],
        "timestamp": row["datetime"].strftime('%Y-%m-%d %H:%M:%S'),
        "event": row["event"],
        "transactionid": row["transactionid"] if pd.notna(row["transactionid"]) else ""
    }

df_events["event_info"] = df_events.apply(extract_event, axis=1)

# 사용자(visitorid)별로 그룹화하고, 시간 간격이 3시간(10800초) 이상이면 세션 분리
user_sessions = {}
three_hours = pd.Timedelta(seconds=10800)

for visitor, group in tqdm(df_events.groupby('visitorid')):
    group = group.sort_values('datetime')
    sessions = []
    current_session = []
    prev_time = None
    for _, row in group.iterrows():
        current_time = row['datetime']
        if prev_time is not None and (current_time - prev_time) >= three_hours:
            if current_session:
                sessions.append(current_session)
            current_session = []
        current_session.append(row["event_info"])
        prev_time = current_time
    if current_session:
        sessions.append(current_session)
    user_sessions[visitor] = sessions

# JSON 파일로 저장 (각 사용자별 세션을 포함)
with open("retailrocket_user_interactions.json", "w", encoding="utf-8") as f:
    json.dump(user_sessions, f, ensure_ascii=False, indent=2)

print("User interactions JSON이 'retailrocket_user_interactions.json' 파일로 저장되었습니다.")


100%|██████████| 1407580/1407580 [05:50<00:00, 4010.43it/s]


User interactions JSON이 'retailrocket_user_interactions.json' 파일로 저장되었습니다.


In [12]:
import json

# 기존 interaction JSON 파일 로드
with open("retailrocket_user_interactions.json", "r", encoding="utf-8") as f:
    user_sessions = json.load(f)

filtered_user_sessions = {}

for user, sessions in user_sessions.items():
    total_interactions = sum(len(session) for session in sessions)
    # 사용자별 세션 수가 3개 이상이고, 총 interaction 수가 5회 이상인 사용자만 선택
    if len(sessions) >= 2 and total_interactions >= 3:
        filtered_user_sessions[user] = sessions

# 필터링 결과를 새로운 JSON 파일로 저장
with open("retailrocket_user_interactions_filtered.json", "w", encoding="utf-8") as f:
    json.dump(filtered_user_sessions, f, ensure_ascii=False, indent=2)

print("Filtered interactions saved to 'retailrocket_user_interactions_filtered.json'.")


Filtered interactions saved to 'retailrocket_user_interactions_filtered.json'.


In [5]:
import json
import statistics

# JSON 파일 읽기
with open("retailrocket_user_interactions.json", "r", encoding="utf-8") as f:
    user_sessions = json.load(f)

# 사용자 수
num_users = len(user_sessions)

# 사용자별 세션 수와 전체 세션별 interaction 수 목록 계산
session_counts = []         # 각 사용자별 세션 수
interaction_counts = []     # 각 세션의 interaction 수

for user, sessions in tqdm(user_sessions.items()):
    session_counts.append(len(sessions))
    for session in sessions:
        interaction_counts.append(len(session))

# 사용자당 세션 수의 평균 계산
avg_sessions_per_user = sum(session_counts) / num_users if num_users > 0 else 0

# 각 세션당 interaction 수의 평균과 분산 계산
if interaction_counts:
    avg_interactions = statistics.mean(interaction_counts)
    var_interactions = statistics.variance(interaction_counts) if len(interaction_counts) > 1 else 0
else:
    avg_interactions = 0
    var_interactions = 0

print("Number of users:", num_users)
print("Average sessions per user:", avg_sessions_per_user)
print("Average interactions per session:", avg_interactions)
print("Variance of interactions per session:", var_interactions)


100%|██████████| 1407580/1407580 [00:00<00:00, 3024449.11it/s]


Number of users: 1407580
Average sessions per user: 1.196835703832109
Average interactions per session: 1.6360158419414925
Variance of interactions per session: 11.471367550718051


In [13]:
import json
import statistics

# JSON 파일 읽기
with open("retailrocket_user_interactions_filtered.json", "r", encoding="utf-8") as f:
    user_sessions = json.load(f)

# 사용자 수
num_users = len(user_sessions)

# 사용자별 세션 수와 전체 세션별 interaction 수 목록 계산
session_counts = []         # 각 사용자별 세션 수
interaction_counts = []     # 각 세션의 interaction 수

for user, sessions in tqdm(user_sessions.items()):
    session_counts.append(len(sessions))
    for session in sessions:
        interaction_counts.append(len(session))

# 사용자당 세션 수의 평균 계산
avg_sessions_per_user = sum(session_counts) / num_users if num_users > 0 else 0

# 각 세션당 interaction 수의 평균과 분산 계산
if interaction_counts:
    avg_interactions = statistics.mean(interaction_counts)
    var_interactions = statistics.variance(interaction_counts) if len(interaction_counts) > 1 else 0
else:
    avg_interactions = 0
    var_interactions = 0

print("Number of users:", num_users)
print("Average sessions per user:", avg_sessions_per_user)
print("Average interactions per session:", avg_interactions)
print("Variance of interactions per session:", var_interactions)


100%|██████████| 96057/96057 [00:00<00:00, 1001074.04it/s]


Number of users: 96057
Average sessions per user: 3.2782618653507813
Average interactions per session: 2.679044140997142
Variance of interactions per session: 49.42560094597455
